In [ ]:
# clone the ADIS repository
!git clone https://github.com/sathishkumar67/SSD_MobileNetV3_ADIS.git
# move the files to the current directory
!mv /kaggle/working/SSD_MobileNetV3_ADIS/* /kaggle/working/
# upgrade pip
!pip install --upgrade pip
# install the required packages
!pip install  -r requirements.txt --upgrade --upgrade-strategy eager

In [2]:
# necessary imports
from typing import List
import os
import optuna
import joblib
from tqdm import tqdm
import random
import numpy as np
from tqdm import tqdm
from huggingface_hub import hf_hub_download
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, RandomSampler
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import LinearLR, CosineAnnealingLR, SequentialLR
from ssdlite_mobnetv3_adis.utils import unzip_file, replace_activation_function
from ssdlite_mobnetv3_adis.dataset import collate_fn, SSDLITEOBJDET_DATASET, CachedSSDLITEOBJDET_DATASET
from ssdlite_mobnetv3_adis.model import SSDLITE_MOBILENET_V3_Large
from ssdlite_mobnetv3_adis.trainer import bohb_tunner, train

# set random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# set constants
REPO_ID = "pt-sk/ADIS" 
DATASET_NAME = "balanced_dataset"
REPO_TYPE = "dataset"
FILENAME_IN_REPO = f"{DATASET_NAME}.zip"
LOCAL_DIR = os.getcwd()
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
DATASET_FOLDER_PATH = f"{LOCAL_DIR}/{DATASET_NAME}"                       
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', 'Squirrel', 'Eagle', 'Goat', 'Rodents', 'Snake'] 
NUM_CLASSES = len(CLASSES)
NUM_CLASSES_WITH_BG = NUM_CLASSES + 1    # 1 for background class

# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)

balanced_dataset.zip:   0%|          | 0.00/7.04G [00:00<?, ?B/s]

Unzipping: 100%|██████████| 7.07G/7.07G [00:41<00:00, 168MB/s]



Unzipped /kaggle/working/balanced_dataset.zip to /kaggle/working
Removed zip file: /kaggle/working/balanced_dataset.zip
Removed zip file: /kaggle/working/balanced_dataset.zip


In [4]:
# set pin memory device
PIN_MEMORY_DEVICE = "cuda:0"
NUM_CORES = os.cpu_count()
BATCH_SIZE = 64

# prepare the dataset
train_dataset = CachedSSDLITEOBJDET_DATASET(
    dataset_class=SSDLITEOBJDET_DATASET,
    root_dir=DATASET_FOLDER_PATH,
    split="train",
    num_classes=NUM_CLASSES_WITH_BG)

val_dataset = CachedSSDLITEOBJDET_DATASET(
    dataset_class=SSDLITEOBJDET_DATASET,
    root_dir=DATASET_FOLDER_PATH,
    split="val",
    num_classes=NUM_CLASSES_WITH_BG)

test_dataset = CachedSSDLITEOBJDET_DATASET(
    dataset_class=SSDLITEOBJDET_DATASET,
    root_dir=DATASET_FOLDER_PATH,
    split="test",
    num_classes=NUM_CLASSES_WITH_BG)


# samplers for reproducibility
train_sampler = RandomSampler(train_dataset, generator=torch.Generator().manual_seed(RANDOM_SEED))
val_sampler = RandomSampler(val_dataset, generator=torch.Generator().manual_seed(RANDOM_SEED))
test_sampler = RandomSampler(test_dataset, generator=torch.Generator().manual_seed(RANDOM_SEED))


# prepare the dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=train_sampler,
    num_workers=NUM_CORES,
    collate_fn=collate_fn,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
    pin_memory_device=PIN_MEMORY_DEVICE)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    sampler=val_sampler,
    num_workers=NUM_CORES,
    collate_fn=collate_fn,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
    pin_memory_device=PIN_MEMORY_DEVICE)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    sampler=test_sampler,
    num_workers=NUM_CORES,
    collate_fn=collate_fn,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=2,
    pin_memory_device=PIN_MEMORY_DEVICE)

Preprocessing dataset and caching to /kaggle/working/balanced_dataset/train_cache...


100%|██████████| 18139/18139 [03:42<00:00, 81.34it/s] 



Preprocessing dataset and caching to /kaggle/working/balanced_dataset/val_cache...


100%|██████████| 2390/2390 [00:25<00:00, 92.03it/s] 


Preprocessing dataset and caching to /kaggle/working/balanced_dataset/test_cache...


 74%|███████▎  | 1757/2390 [00:20<00:08, 78.71it/s] libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
100%|██████████| 2390/2390 [00:27<00:00, 86.36it/s] 


In [5]:
custom_ckpt = torch.load("/kaggle/working/ckpt/ssdlite_mobv3_custom_params_ckpt.pth", map_location="cpu")
best_ckpt = torch.load("/kaggle/working/ckpt/ssdlite_mobnetv3_bestparams_ckpt.pth", map_location="cpu")

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model = SSDLITE_MOBILENET_V3_Large(num_classes_with_bg=NUM_CLASSES_WITH_BG)
custom_model.load_state_dict(custom_ckpt['model_state_dict'], strict=True)
custom_model.to(device)
custom_model.eval()

best_model = SSDLITE_MOBILENET_V3_Large(num_classes_with_bg=NUM_CLASSES_WITH_BG)
best_model.load_state_dict(best_ckpt['model_state_dict'], strict=True)
best_model.to(device)
best_model.eval()

print("Model loaded successfully!")

Downloading: "https://download.pytorch.org/models/ssdlite320_mobilenet_v3_large_coco-a79551df.pth" to /root/.cache/torch/hub/checkpoints/ssdlite320_mobilenet_v3_large_coco-a79551df.pth
100%|██████████| 13.4M/13.4M [00:02<00:00, 4.97MB/s]
100%|██████████| 13.4M/13.4M [00:02<00:00, 4.97MB/s]


Model loaded successfully!


In [ ]:
# import time
# start_time = time.time()
# df_metrics = calculate_per_class_with_iou(best_model, train_loader, device, classes=CLASSES)
# df_metrics.loc["Average"] = df_metrics.mean()
# print(f"Per-class metrics for test set:\n{df_metrics}")
# end_time = time.time()
# print(f"Time taken for test set evaluation: {end_time - start_time:.2f} seconds") 

In [33]:
# compute per class metrics for validation set
# compute_per_class_metrics(best_model, val_loader, device, class_names=CLASSES)

In [54]:
compute_average_metrics(best_model, val_loader, device, dataset_name="ValidationSet", class_names=CLASSES)

TypeError: compute_average_metrics() got multiple values for argument 'dataset_name'

In [ ]:
from __future__ import annotations
import time
import pandas as pd
from collections import defaultdict
from torchvision.ops import box_iou
from torchmetrics.detection.mean_ap import MeanAveragePrecision

# compute and print average metrics for a model and dataloader
def compute_average_metrics(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    device: torch.device,
    class_names: list,
    conf_thresh: float = 0.2,
    iou_thresh: float = 0.5
    ) -> pd.DataFrame:
    """
    Computes per-class metrics (accuracy, precision, recall, f1-score, IoU, mAP@50, mAP@50:95) for a model and dataloader.

    Args:
        model (torch.nn.Module): Trained model for evaluation.
        dataloader (torch.utils.data.DataLoader): DataLoader for evaluation.
        device (torch.device): Device to run model on.
        class_names (list): List of class names for DataFrame index.
        conf_thresh (float, optional): Confidence threshold for predictions. Defaults to 0.2.
        iou_thresh (float, optional): IoU threshold for matching. Defaults to 0.5.

    Returns:
        pd.DataFrame: DataFrame with per-class metrics.
    """

    print("[Stage 1] Initializing metric containers and TorchMetrics...")

    counters = defaultdict(lambda: {"tp":0,"fp":0,"fn":0,"support":0})
    iou_sums = defaultdict(float)
    iou_counts = defaultdict(int)

    metric = MeanAveragePrecision(
        box_format='xyxy',
        iou_type='bbox',
        iou_thresholds=[0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95],
        class_metrics=True,
        extended_summary=True
    )

    print("[Stage 2] Starting inference and metric collection...")
    start_time = time.time()
    model.eval()

    with torch.no_grad():
        for images, targets in tqdm(dataloader, desc="Inference", leave=True):

            # Move images to device
            images = images.to(device) if isinstance(images, torch.Tensor) else [img.to(device) for img in images]
            outputs = model(images)

            preds = []
            targs = []

            for out, tgt in zip(outputs, targets):

                # Prepare TorchMetrics format
                preds.append({
                    'boxes': out['boxes'].cpu(),
                    'scores': out['scores'].cpu(),
                    'labels': out['labels'].cpu()
                })
                targs.append({
                    'boxes': tgt['boxes'].cpu(),
                    'labels': tgt['labels'].cpu()
                })

                # Per-class metrics
                pred_boxes = out["boxes"].cpu()
                pred_scores = out["scores"].cpu()
                pred_labels = out["labels"].cpu()
                true_boxes = tgt["boxes"]
                true_labels = tgt["labels"]

                # Filter by confidence
                keep = pred_scores > conf_thresh
                pred_boxes = pred_boxes[keep]
                pred_labels = pred_labels[keep]

                # Count support (number of GT instances per class)
                for lbl in true_labels.tolist():
                    counters[lbl]["support"] += 1

                # No predictions → all GT are FN
                if pred_boxes.numel() == 0:
                    for lbl in true_labels.tolist():
                        counters[lbl]["fn"] += 1
                    continue

                # Compute IoU matrix and find matches
                iou_matrix = box_iou(pred_boxes, true_boxes)
                matches = torch.nonzero(iou_matrix > iou_thresh, as_tuple=False)
                matched_pred, matched_true = set(), set()

                for pi, ti in matches.tolist():
                    matched_pred.add(pi)
                    matched_true.add(ti)
                    p_lbl = int(pred_labels[pi].item())
                    t_lbl = int(true_labels[ti].item())
                    if p_lbl == t_lbl:
                        counters[p_lbl]["tp"] += 1
                        iou_sums[p_lbl] += iou_matrix[pi, ti].item()
                        iou_counts[p_lbl] += 1
                    else:
                        counters[p_lbl]["fp"] += 1
                        counters[t_lbl]["fn"] += 1

                # Unmatched → FP or FN
                for pi in range(len(pred_boxes)):
                    if pi not in matched_pred:
                        cls = int(pred_labels[pi].item())
                        counters[cls]["fp"] += 1
                for ti in range(len(true_boxes)):
                    if ti not in matched_true:
                        cls = int(true_labels[ti].item())
                        counters[cls]["fn"] += 1

            metric.update(preds, targs)

    print("[Stage 3] Computing per-class metrics and building DataFrame...")

    results = {}
    for cls, cnt in counters.items():
        tp, fp, fn, sup = cnt["tp"], cnt["fp"], cnt["fn"], cnt["support"]
        prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1   = 2*prec*rec/(prec+rec) if (prec+rec) > 0 else 0.0
        avg_iou = iou_sums[cls]/iou_counts[cls] if iou_counts[cls]>0 else 0.0
        accuracy = tp / (tp + fp + fn) if (tp + fp + fn) > 0 else 0.0
        results[cls] = {
            "count": sup,
            "precision": prec,
            "recall": rec,
            "f1_score": f1,
            "accuracy": accuracy,
            "avg_iou": avg_iou
        }

    df_metrics = pd.DataFrame(results).T

    # Set class names as index
    df_metrics.index = [class_names[idx-1] for idx in df_metrics.index]
    df_metrics = df_metrics.sort_index()

    # Add mAP columns from TorchMetrics
    tm_results = metric.compute()
    if "map_50" in tm_results:
        df_metrics["mAP@50"] = tm_results["map_50"].cpu().tolist()
    if "map" in tm_results:
        df_metrics["mAP@[50:95]"] = tm_results["map"].cpu().tolist()

    # Add last row for average metrics
    df_metrics.loc["Average"] = df_metrics.mean()

    print("[Stage 4] Average Metrics:")
    print(df_metrics.T["Average"])

    end_time = time.time()
    print(f"[Stage 5] Evaluation completed. Time taken: {end_time - start_time:.2f} seconds.")

    return df_metrics